In [2]:
import neurokit2 as nk
import numpy as np
import NeuroHEP as hep

## Generate synthetic ECG and EEG signals

In [ ]:
# For 5 subjects (10 seconds of data at 250 Hz)
num_subjects = 5
sampling_rate = 250  # Hz
duration = 10  # seconds

# Simulating ECG and EEG signals
ecg_signals = np.array([nk.ecg_simulate(duration=duration, sampling_rate=sampling_rate, heart_rate=70) for _ in range(num_subjects)])
eeg_signals = np.array([nk.eeg_simulate(duration=duration, sampling_rate=sampling_rate, noise=0.1) for _ in range(num_subjects)])

## Pre-processing

In [ ]:
# Desired parameters for preprocessing
target_fs = 128  # Resample to 128 Hz
notch_freq = 60  # 60 Hz notch filter for powerline noise
low_cutoff = 0.1  # Low cutoff for bandpass filter (in Hz)
high_cutoff = 35  # High cutoff for bandpass filter (in Hz)

# Preprocess ECG and EEG signals
filtered_ecg, filtered_eeg = hep.preprocessing.preprocess_ecg_eeg(
    ecg_signals, eeg_signals, sampling_rate, 
    target_fs=target_fs, notch_freq=notch_freq, 
    low_cutoff=low_cutoff, high_cutoff=high_cutoff
)

## Extract HEPs

In [ ]:
# Select one subject's ECG signal for R-wave peak detection (for simplicity, using the first subject)
subject_ecg_signal = filtered_ecg[0]
subject_eeg_signal = filtered_eeg[0]

In [ ]:
# Detect R-wave peaks
r_peak_indices = hep.detection.detect_r_wave_peaks(subject_ecg_signal, sampling_rate)

In [ ]:
# Define windows around R-peaks (in milliseconds)
pre_window_ms = 100  # 100 ms before the R-peak
post_window_ms = 300  # 300 ms after the R-peak

# Extract Heart-Evoked Potentials (HEPs)
hep, hep_ecg = hep.detection.extract_heps(
    subject_eeg_signal, subject_ecg_signal, r_peak_indices, 
    pre_window_ms, post_window_ms, sampling_rate, show_plots=False
)

## Visualizaiton

In [ ]:
hep.visualization.plot_ecg_eeg_with_hep()